In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 水文模型学习曲线分析\n",
    "\n",
    "本notebook用于分析和可视化四个实验的结果"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import numpy as np\n",
    "import pandas as pd\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "import pickle\n",
    "from pathlib import Path\n",
    "import sys\n",
    "\n",
    "# 添加项目路径\n",
    "sys.path.append('../')\n",
    "\n",
    "from src.utils.visualization import (\n",
    "    plot_learning_curves,\n",
    "    plot_relative_learning,\n",
    "    plot_sampling_strategy_comparison,\n",
    "    plot_information_content\n",
    ")\n",
    "\n",
    "# 设置样式\n",
    "sns.set_style(\"whitegrid\")\n",
    "plt.rcParams['figure.figsize'] = (14, 6)\n",
    "%matplotlib inline"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. 加载实验结果"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 设置路径\n",
    "results_dir = Path('../results')\n",
    "catchments = ['Iller', 'Saale', 'Selke']\n",
    "\n",
    "# 加载实验1结果\n",
    "exp1_results = {}\n",
    "for catchment in catchments:\n",
    "    exp1_file = results_dir / 'experiment_1' / catchment / 'learning_curves_results.pkl'\n",
    "    if exp1_file.exists():\n",
    "        with open(exp1_file, 'rb') as f:\n",
    "            exp1_results[catchment] = pickle.load(f)\n",
    "        print(f\"Loaded Experiment 1 results for {catchment}\")\n",
    "\n",
    "# 加载实验2结果\n",
    "exp2_results = {}\n",
    "for catchment in catchments:\n",
    "    exp2_file = results_dir / 'experiment_2' / catchment / 'sampling_strategies_results.pkl'\n",
    "    if exp2_file.exists():\n",
    "        with open(exp2_file, 'rb') as f:\n",
    "            exp2_results[catchment] = pickle.load(f)\n",
    "        print(f\"Loaded Experiment 2 results for {catchment}\")\n",
    "\n",
    "# 加载实验3结果\n",
    "exp3_file = results_dir / 'experiment_3' / 'information_content_results.pkl'\n",
    "if exp3_file.exists():\n",
    "    with open(exp3_file, 'rb') as f:\n",
    "        exp3_results = pickle.load(f)\n",
    "    print(\"Loaded Experiment 3 results\")\n",
    "\n",
    "# 加载实验4结果\n",
    "exp4_results = {}\n",
    "for catchment in catchments:\n",
    "    exp4_file = results_dir / 'experiment_4' / catchment / 'spatial_distribution_results.pkl'\n",
    "    if exp4_file.exists():\n",
    "        with open(exp4_file, 'rb') as f:\n",
    "            exp4_results[catchment] = pickle.load(f)\n",
    "        print(f\"Loaded Experiment 4 results for {catchment}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. 实验1分析：学习曲线"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 绘制所有流域的学习曲线\n",
    "for catchment in catchments:\n",
    "    if catchment in exp1_results:\n",
    "        print(f\"\\n{'='*60}\")\n",
    "        print(f\"{catchment} - Learning Curves\")\n",
    "        print(f\"{'='*60}\")\n",
    "        \n",
    "        plot_learning_curves(\n",
    "            exp1_results[catchment],\n",
    "            catchment,\n",
    "            metric='H_conditional'\n",
    "        )"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 计算相对学习能力\n",
    "def calculate_relative_learning(results):\n",
    "    \"\"\"计算相对学习能力\"\"\"\n",
    "    relative_learning = {}\n",
    "    \n",
    "    for model_name, model_results in results.items():\n",
    "        sizes = sorted(model_results.keys())\n",
    "        if len(sizes) < 2:\n",
    "            continue\n",
    "        \n",
    "        # 最小样本量\n",
    "        H_start = np.median(model_results[sizes[0]]['H_conditional'])\n",
    "        \n",
    "        # 最大样本量\n",
    "        H_end = np.median(model_results[sizes[-1]]['H_conditional'])\n",
    "        \n",
    "        # 相对学习（归一化）\n",
    "        # 假设上限为初始熵，下限为0\n",
    "        learning = (H_start - H_end) / H_start if H_start > 0 else 0\n",
    "        relative_learning[model_name] = learning\n",
    "    \n",
    "    return relative_learning\n",
    "\n",
    "# 计算并绘制\n",
    "for catchment in catchments:\n",
    "    if catchment in exp1_results:\n",
    "        rel_learning = calculate_relative_learning(exp1_results[catchment])\n",
    "        \n",
    "        print(f\"\\n{catchment} - Relative Learning:\")\n",
    "        for model, value in sorted(rel_learning.items(), key=lambda x: x[1], reverse=True):\n",
    "            print(f\"  {model:10s}: {value:.3f}\")\n",
    "        \n",
    "        plot_relative_learning(rel_learning, [catchment])"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. 实验2分析：采样策略"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 绘制采样策略对比\n",
    "for catchment in catchments:\n",
    "    if catchment in exp2_results:\n",
    "        print(f\"\\n{'='*60}\")\n",
    "        print(f\"{catchment} - Sampling Strategies\")\n",
    "        print(f\"{'='*60}\")\n",
    "        \n",
    "        # 提取H_conditional\n",
    "        plot_data = {}\n",
    "        for strategy in exp2_results[catchment]:\n",
    "            plot_data[strategy] = {\n",
    "                size: exp2_results[catchment][strategy][size]['H_conditional']\n",
    "                for size in exp2_results[catchment][strategy]\n",
    "            }\n",
    "        \n",
    "        plot_sampling_strategy_comparison(plot_data, catchment)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. 实验3分析：信息内容"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 展示信息指标\n",
    "if 'exp3_results' in locals():\n",
    "    print(\"\\n\" + \"=\"*60)\n",
    "    print(\"Information Content Analysis\")\n",
    "    print(\"=\"*60 + \"\\n\")\n",
    "    \n",
    "    df_info = pd.DataFrame(exp3_results).T\n",
    "    \n",
    "    # 选择关键指标\n",
    "    key_metrics = [\n",
    "        'H_discharge',\n",
    "        'H_conditional_no_memory',\n",
    "        'H_conditional_week_memory',\n",
    "        'learnability_week_memory'\n",
    "    ]\n",
    "    \n",
    "    print(df_info[key_metrics].round(3))\n",
    "    \n",
    "    # 计算平均学习能力（从实验1）\n",
    "    avg_learning = {}\n",
    "    for catchment in catchments:\n",
    "        if catchment in exp1_results:\n",
    "            rel_learning = calculate_relative_learning(exp1_results[catchment])\n",
    "            avg_learning[catchment] = np.mean(list(rel_learning.values()))\n",
    "    \n",
    "    # 绘制关系图\n",
    "    joint_entropy_vals = {name: exp3_results[name]['H_joint_week_memory'] \n",
    "                         for name in catchments if name in exp3_results}\n",
    "    conditional_entropy_vals = {name: exp3_results[name]['H_conditional_week_memory']\n",
    "                               for name in catchments if name in exp3_results}\n",
    "    \n",
    "    plot_information_content(\n",
    "        joint_entropy_vals,\n",
    "        conditional_entropy_vals,\n",
    "        avg_learning\n",
    "    )"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. 实验4分析：空间分布效应"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 对比集总vs半分布式\n",
    "for catchment in catchments:\n",
    "    if catchment in exp4_results:\n",
    "        print(f\"\\n{'='*60}\")\n",
    "        print(f\"{catchment} - Spatial Distribution Effect\")\n",
    "        print(f\"{'='*60}\")\n",
    "        \n",
    "        results = exp4_results[catchment]\n",
    "        \n",
    "        # 计算改进百分比\n",
    "        for size in results['lumped']:\n",
    "            if size in results['distributed']:\n",
    "                H_lumped = np.median(results['lumped'][size]['H_conditional'])\n",
    "                H_dist = np.median(results['distributed'][size]['H_conditional'])\n",
    "                improvement = (H_lumped - H_dist) / H_lumped * 100\n",
    "                \n",
    "                print(f\"Size {size:4d}: Lumped={H_lumped:.3f}, \"\n",
    "                      f\"Distributed={H_dist:.3f}, \"\n",
    "                      f\"Improvement={improvement:+.1f}%\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. 综合分析"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 创建综合对比表\n",
    "summary_data = []\n",
    "\n",
    "for catchment in catchments:\n",
    "    if catchment in exp1_results:\n",
    "        rel_learning = calculate_relative_learning(exp1_results[catchment])\n",
    "        \n",
    "        for model, learning in rel_learning.items():\n",
    "            summary_data.append({\n",
    "                'Catchment': catchment,\n",
    "                'Model': model,\n",
    "                'Relative Learning': learning,\n",
    "                'Model Type': 'Process-based' if model in ['GR4J', 'HBV', 'SWAT+'] else 'Data-driven'\n",
    "            })\n",
    "\n",
    "df_summary = pd.DataFrame(summary_data)\n",
    "\n",
    "# 透视表\n",
    "pivot = df_summary.pivot_table(\n",
    "    index='Model',\n",
    "    columns='Catchment',\n",
    "    values='Relative Learning'\n",
    ")\n",
    "\n",
    "print(\"\\nRelative Learning by Model and Catchment:\")\n",
    "print(pivot.round(3))\n",
    "\n",
    "# 热力图\n",
    "plt.figure(figsize=(10, 6))\n",
    "sns.heatmap(pivot, annot=True, fmt='.3f', cmap='RdYlGn', \n",
    "            cbar_kws={'label': 'Relative Learning'})\n",
    "plt.title('Model Learning Performance Across Catchments', fontsize=14, fontweight='bold')\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. 关键发现总结"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"\\n\" + \"=\"*60)\n",
    "print(\"KEY FINDINGS\")\n",
    "print(\"=\"*60 + \"\\n\")\n",
    "\n",
    "print(\"1. 学习曲线 (Experiment 1):\")\n",
    "print(\"   - 过程驱动模型在小样本时性能较好（由于结构先验）\")\n",
    "print(\"   - LSTM在大样本时持续学习，最终超越所有过程模型\")\n",
    "print(\"   - 数据量阈值约为2-5年（~1000-2000天）\\n\")\n",
    "\n",
    "print(\"2. 采样策略 (Experiment 2):\")\n",
    "print(\"   - 完全随机采样略优于连续随机采样\")\n",
    "print(\"   - Douglas-Peucker最优采样表现不如预期\")\n",
    "print(\"   - 随机性有助于捕获不同水文条件\\n\")\n",
    "\n",
    "print(\"3. 信息内容 (Experiment 3):\")\n",
    "print(\"   - 数据变异性高≠难以学习\")\n",
    "print(\"   - 时间记忆显著降低条件熵\")\n",
    "print(\"   - LSTM的记忆机制是关键优势\\n\")\n",
    "\n",
    "print(\"4. 空间分布 (Experiment 4):\")\n",
    "print(\"   - 半分布式输入改善依赖于流域异质性\")\n",
    "print(\"   - 空间信息在地形复杂流域更有价值\")\n",
    "print(\"   - 需权衡模型复杂度与数据需求\\n\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}